# Phase 1: Tropical Cyclone Damage Assesment

## Challenge Phase 1 Overview

In [5]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
#from osgeo import gdal

# Visualisation
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
from ultralytics import YOLO
#import yolov9
import labelme2yolo

# Others
import os
import shutil
import zipfile
import cv2
import numpy as np




## Data Pre Processing 

### Tiles generation

This function generate tiles from an image. It can be used on only a part of the image, defined by start_x_prop, start_y_prop, end_x_prop and end_y_prop.

In [2]:
def generate_tiles(input_file, output_dir, grid_x, grid_y, start_x_prop, start_y_prop, end_x_prop, end_y_prop, id):
    ds = gdal.Open(input_file)


    # Get image size and number of bands
    width = ds.RasterXSize
    height = ds.RasterYSize
    num_bands = ds.RasterCount

    start_x = int(start_x_prop*width)
    end_x = int(end_x_prop*width)
    start_y = int(start_y_prop*height)
    end_y = int(end_y_prop*height)



    # Calculate number of tiles in each dimension within the specified area
    num_tiles_x = ((end_x - start_x) // grid_x) + 1
    num_tiles_y = ((end_y - start_y) // grid_y) + 1

    print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over each tile and save as a separate TIFF image
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            x_offset = start_x + (i * grid_x)
            y_offset = start_y + (j * grid_y)

            # Ensure tile doesn't exceed image bounds
            if x_offset >= end_x or y_offset >= end_y:
                continue

            tile_width = min(grid_x, end_x - x_offset)
            tile_height = min(grid_y, end_y - y_offset)

            tile = []
            for band in range(1, num_bands + 1):
                tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
                tile.append(tile_data)

            # Create output filename
            output_file = os.path.join(output_dir, f"tile_{id}_{i}_{j}.tif")

            # Create an output TIFF file with same CRS and band values range
            driver = gdal.GetDriverByName("GTiff")
            options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
            out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                                    ds.GetRasterBand(1).DataType, options=options)

            # Set the geotransform
            geotransform = list(ds.GetGeoTransform())
            geotransform[0] = geotransform[0] + x_offset * geotransform[1]
            geotransform[3] = geotransform[3] + y_offset * geotransform[5]
            out_ds.SetGeoTransform(tuple(geotransform))

            # Set the projection
            out_ds.SetProjection(ds.GetProjection())

            # Write each band to the output file
            for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])

            # Close the output file
            out_ds = None

    print("Tiles generation completed.")

In [3]:
input_file = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_San_Juan.tif"
output_dir_1 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_TIFF_1"
output_dir_2 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_TIFF_2"
output_dir_3 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_TIFF_3"


grid_x = 512
grid_y = 512
generate_tiles(input_file, output_dir_1,grid_x,grid_y, 0.333, 0, 1, 0.333, 1)
generate_tiles(input_file, output_dir_2,grid_x,grid_y, 0, 0.8, 1, 1, 2)
generate_tiles(input_file, output_dir_3,grid_x,grid_y, 0, 0.25, 1, 0.5, 3)

Total number of tiles: 2775
Tiles generation completed.


## Labelling

This function is used to convert TIFF images to JPEG images to label them with LabelMe.

In [19]:
def convert_tiff_to_jpeg(input_dir,output_dir):
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(input_dir, filename))
        
            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')
        
            # create new filename, replace .tif with .jpg
            output_filename = os.path.splitext(filename)[0] + '.jpg'
        
            # save the image in JPEG format
            img.save(os.path.join(output_dir, output_filename), 'JPEG')
    print("Conversion from TIFF to JPEG completed.")

In [20]:
# specify directory
input_dir_1 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_TIFF_1"
output_dir_1 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_JPEG_1"
input_dir_2 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_TIFF_2"
output_dir_2 = "/Users/homefolder/Desktop/Challenge_EY/Post_Event_Grids_In_JPEG_2"
input_dir_3 = './Post_Event_Grids_In_TIFF_3'
output_dir_3 = './Post_Event_Grids_In_JPEG_3'

convert_tiff_to_jpeg(input_dir_1,output_dir_1)
convert_tiff_to_jpeg(input_dir_2,output_dir_2)
convert_tiff_to_jpeg(input_dir_3,output_dir_3)

Conversion from TIFF to JPEG completed.


### Renaming the Files

<div align = "justify">For easier and more efficient data accessibility, it's necessary to rename the files in the directory. We'll use the function <b><i>rename_files</b></i> to accomplish this task of altering the file names in the given path.</div>

In [31]:
def rename_files(directory_path, id):
# Define the directory path where your files are located
    directory_path = directory_path
    
    # Get a list of all files in the directory
    files = os.listdir(directory_path)
    
    # Define a prefix for the new file names 
    prefix = f"Post_Event_{id}_"
    
    # Start the numbering from 1
    number = 0
    
    # Loop through each file in the directory
    for filename in files:
        # Check if the item is a file (not a directory)
        if os.path.isfile(os.path.join(directory_path, filename)):
            # Get the file extension
            file_extension = os.path.splitext(filename)[1]
    
            # Create the new file name with leading zeros
            new_filename = f"{prefix}{number:03}{file_extension}"
    
            # Construct the full path to the original and new files
            old_filepath = os.path.join(directory_path, filename)
            new_filepath = os.path.join(directory_path, new_filename)
    
            # Rename the file
            os.rename(old_filepath, new_filepath)
    
            # Increment the number for the next file
            number += 1
    
    print("Files renamed successfully.")

In [27]:
rename_files(output_dir_1, 1)
rename_files(output_dir_2, 2)
rename_files(output_dir_3, 3)

Files renamed successfully.
Files renamed successfully.
Files renamed successfully.


## Model Building
<div align="justify">

My model is based on the following idea : divide to conquer. Therefore, it first tries to estimate if the image is from the countryside, the suburbs, or the city center. I believe those three categories are the most prominent ones in the dataset.
The classifition is made not by using a CNN model but simply by finding the proportion of green and brown pixels. It is far from being perfect, but I belive it will still make it easier for the model to learn.

### Image Classification

This function is used to determine if the given image is from the countryside, the suburbs, or the city center. 

In [21]:
def find_area_type(input_folder, filename):

    basename, extension = os.path.splitext(filename)

    # Lire l'image
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)

    # Convertir l'image en espace de couleur HSV
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Définir les plages de couleur verte et marron
    lower_green = np.array([40, 20, 10])
    upper_green = np.array([165, 255, 254])
    lower_brown = np.array([1, 30, 10])
    upper_brown = np.array([40, 255, 254])

    # Créer un masque pour les pixels verts et bruns
    mask_green = cv2.inRange(img_hsv, lower_green, upper_green)
    mask_brown = cv2.inRange(img_hsv, lower_brown, upper_brown)

    total_pixels = img.shape[0] * img.shape[1]

    # Calculer le pourcentage de pixels verts et bruns
    green_percentage = cv2.countNonZero(mask_green) / total_pixels
    brown_percentage = cv2.countNonZero(mask_brown) / total_pixels

    return green_percentage + brown_percentage




This function sorts the images in the given path into folders. 

In [22]:
import cv2
import numpy as np
from shutil import copy

def classify_images(input_folder):
    # Créer les dossiers de sortie s'ils n'existent pas
    
    output_folder_countryside = "./images_countryside"
    output_folder_city = "./images_city"
    output_folder_suburbs = "./images_suburbs"
    os.makedirs(output_folder_countryside, exist_ok=True)
    os.makedirs(output_folder_city, exist_ok=True)
    os.makedirs(output_folder_suburbs, exist_ok=True)


    # Liste des extensions d'images supportées
    img_extensions = ['.jpg', '.jpeg', '.png']

    # Parcourir les fichiers dans le dossier d'entrée
    for filename in os.listdir(input_folder):
        basename, extension = os.path.splitext(filename)
        if extension.lower() in img_extensions:
            
            img_path = os.path.join(input_folder, filename)
            percentage = find_area_type(input_folder, filename)
            
            # Déterminer la catégorie de l'image
            if percentage > 0.80 :
                output_path = os.path.join(output_folder_countryside, filename)
            elif percentage > 0.5 :
                output_path = os.path.join(output_folder_suburbs, filename)
            else:
                output_path = os.path.join(output_folder_city, filename)

            # Copier l'image dans le dossier de sortie correspondant
            copy(img_path, output_path)

            # Copier également le fichier .json s'il existe
            json_path = os.path.join(input_folder, f"{basename}.json")
            if os.path.exists(json_path):
                copy(json_path, os.path.dirname(output_path))
    print('Done')



In [23]:
classify_images('./Post_Event_Grids_In_JPEG_1/')
classify_images('./Post_Event_Grids_In_JPEG_2/')

Done
Done


### Labelme to Yolo

In [ ]:
def labelme_to_yolo(source, target_dir):
    source_images_train = os.path.join(source, "images/train")
    source_images_val = os.path.join(source, "images/val")
    source_labels_train = os.path.join(source, "labels/train")
    source_labels_val = os.path.join(source, "labels/val")
    source_yaml = os.path.join(source, "dataset.yaml")
    

    target_images_train = os.path.join(target_dir, "train")
    target_images_val = os.path.join(target_dir, "val")

    # Supprimer les fichiers dans le répertoire cible
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)

    # Créer les répertoires train et val
    os.makedirs(target_images_train)
    os.makedirs(target_images_val)

    # Déplacer les fichiers d'images
    for src, dest in [(source_images_train, target_images_train), (source_images_val, target_images_val)]:
        for filename in os.listdir(src):
            shutil.move(os.path.join(src, filename), dest)

    # Déplacer les fichiers de labels
    for src, dest in [(source_labels_train, target_images_train), (source_labels_val, target_images_val)]:
        for filename in os.listdir(src):
            shutil.move(os.path.join(src, filename), dest)
    


I also decided to use two models instead of one. The first one is used to identify the residential and the commercial area and the second one is used to identify the damaged building inside those areas. Since I made the choice to classify the images into three categories, there are therefore six models. It does increase the number of data that needs to be labelled by two but the labelling for the first one is actually quite easy. Note that if the test images weren't 512x512 I could've trained my first model with bigger images which would have been very beneficial since the residential and commercial areas are easier to identify on a bigger scale.

## Model 1

### Suburbs


In [ ]:
!labelme2yolo --json_dir ./images_suburbs

# Chemins des répertoires et fichiers
source = "./images_suburbs_verif/YOLODataset/images/"
target_dir = "./datasets/datasets_suburbs"

labelme_to_yolo(source, target_dir)

In [10]:
#model_suburbs = YOLO('./runs/detect/train76/weights/best.pt')
model_suburbs = YOLO('runs/detect/train18/weights/best.pt')

In [ ]:
model_suburbs.train(data='dataset_suburbs.yaml', epochs=300, imgsz=512, mixup=0.7, fliplr=0.5, flipud=0.5, copy_paste=0.5, auto_augment='autoaugment',patience=0) 

### City


In [ ]:
!labelme2yolo --json_dir ./images_city

# Chemins des répertoires et fichiers

source = "./images_city/YOLODataset/"
target_dir = "./datasets/datasets_city"

labelme_to_yolo(source, target_dir)


In [23]:
#Loading the model
model_city = YOLO('./runs/detect/train40/weights/best.pt')


YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [ ]:
model_city.train(data='./dataset_city.yaml', epochs=700, imgsz=512)

### Countryside


In [148]:
!labelme2yolo --json_dir ./images_countryside

# Chemins des répertoires et fichiers

source = "./images_countryside/YOLODataset/"
target_dir = "./datasets/images_countryside"

labelme_to_yolo(source, target_dir)


INFO:labelme2yolo:Converting train set ...
Converting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00m 0:00:01
INFO:labelme2yolo:Converting val set ...
Converting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00m 0:00:01
INFO:labelme2yolo:Converting test set ...
Converting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--


In [158]:
model_countryside = YOLO('./runs/detect/train46/weights/best.pt')

Model summary: 225 layers, 3011238 parameters, 0 gradients, 8.2 GFLOPs


(225, 3011238, 0, 8.1952256)

In [ ]:
results_countryside = model_countryside.train(data='./dataset_countryside.yaml', epochs=30, imgsz=512, save=True)


## Model 1 to Model 2

This function will be used to change the hue of the identified area (residential or commercial). Unfortunatly, changing the hue is making us lose a lot of informations. Note that using RGBA and only changing the alpha channel would allow us to not lose the inforamtions linked to chromiannce but yolo only take in RGB images. However the nulber of channels can be change in the architecture of yolo, I did not have enough time to do it.

In [24]:
import os
import cv2
import numpy as np

def change_hue_val(image, label, points, ext):
    # Convertir l'image en HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Définir les valeurs de teinte pour chaque label
    #if ext=='jpg':
    hue_values = {"residential": 30, "commercial": 150}  # Jaune: 30, Violet: 150
    #else : hue_values = {"0": 30, "1": 150}  # Jaune: 30, Violet: 150
    hue = hue_values[label]

    # Créer un masque pour le polygone
    mask = np.zeros_like(hsv_image[:, :, 0])
    cv2.fillPoly(mask, [points], 255)

    hsv_image[:, :, 0] = np.where(mask == 255, hue, hsv_image[:, :, 0])
    hsv_image[:, :, 1] = np.where(mask == 255, 50, hsv_image[:, :, 1])  # Saturation
    hsv_image[:, :, 2] = np.where(mask == 100, 50, hsv_image[:, :, 2])  # Luminosité


    # Reconvertir l'image en RGB
    result_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    
    return result_image


This function find the pixels that need to be changed and then use the change_hue function to do it.

In [ ]:

def draw_and_save_images_val(image_path, annotation_file, output_dir, ext):
    # Charger l'image
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    # Lire le fichier d'annotations
    with open(annotation_file, 'r') as file:
        annotations = file.readlines()

    # Créer une copie de l'image originale pour conserver les modifications
    result_image = image.copy()

    # Parcourir les annotations
    for annotation in annotations:
        annotation = annotation.strip().split()
        label = annotation[0]
        # Les coordonnées sont normalisées, on les reconvertit
        points = [0,0,0,0]

        points[0] = (int(float(annotation[2])), int(float(annotation[3])))
        points[1] = (int(float(annotation[4])), int(float(annotation[3])))
        points[2] = (int(float(annotation[4])), int(float(annotation[5])))
        points[3] = (int(float(annotation[2])), int(float(annotation[5])))
        
        # Modifier la teinte des pixels dans le polygone
        result_image = change_hue_val(result_image, label, np.array(points), ext)
        
    # Enregistrer l'image avec les modifications
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, result_image)

    


Finnaly this function calls the previous function and uses it on every image in the input folder. 

In [ ]:
def area_identification_val(input_dir, output_dir, ext):
    # Parcourir les fichiers dans le répertoire des données
    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            # Récupérer le chemin de l'image correspondante
            image_path = os.path.join(input_dir, filename[:-4] + '.'+ext)
            annotation_file = os.path.join(input_dir, filename)
            # Vérifier si le fichier image existe et que le fichier d'annotation est valide
            if os.path.exists(image_path):
                # Appliquer les modifications sur l'image et l'enregistrer dans le répertoire de sortie
                draw_and_save_images_val(image_path, annotation_file, output_dir, ext)


In [21]:
def generate_txt(directory, results_directory, ext, models, decoding_of_predictions):
    model1_countryside, model1_suburbs, model1_city = models
    
    if os.path.exists(results_directory) : shutil.rmtree(results_directory)
    os.mkdir(results_directory)
    
    # Loop through each file in the directory
    for filename in os.listdir(directory):
        # Check if the current object is a file and ends with .jpeg
        if os.path.isfile(os.path.join(directory, filename)) and filename.lower().endswith(ext):
            # Perform operations on the file
            file_path = os.path.join(directory, filename)
            print(file_path)
            print("Making a prediction on ", filename)

            percentage = find_area_type(directory, filename)
            
            # Déterminer la catégorie de l'image
            if percentage > 0.80 :
                results = model1_countryside.predict(file_path, save=True, iou=0.5, save_txt=True, conf=0.4)
            elif percentage > 0.5 :
                results = model1_suburbs.predict(file_path, save=True, iou=0.5, save_txt=True, conf=0.4)
            else:
                results = model1_city.predict(file_path, save=True, iou=0.5, save_txt=True, conf=0.4)

            for r in results:
    
                conf_list = r.boxes.conf.cpu().numpy().tolist()
                clss_list = r.boxes.cls.cpu().numpy().tolist()
                original_list = clss_list
                updated_list = []
                for element in original_list:
                        updated_list.append(decoding_of_predictions[int(element)])
    
            bounding_boxes = r.boxes.xyxy.cpu().numpy()
            confidences = conf_list
            class_names = updated_list
    
            # Check if bounding boxes, confidences and class names match
            if len(bounding_boxes) != len(confidences) or len(bounding_boxes) != len(class_names):
                print("Error: Number of bounding boxes, confidences, and class names should be the same.")
                continue
            
            text_file_name = os.path.splitext(filename)[0]
            # Creating a new .txt file for each image in the submission_directory
            with open(os.path.join(results_directory, f"{text_file_name}.txt"), "w") as file:
                for i in range(len(bounding_boxes)):
                    # Get coordinates of each bounding box
                    left, top, right, bottom = bounding_boxes[i]
                    # Write content to file in desired format
                    file.write(f"{class_names[i]} {confidences[i]} {left} {top} {right} {bottom}\n")
            print("Output files generated successfully.")

In [22]:
def copy_images(source_dir, destination_dir, ext):
    
    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        # Vérifie si le fichier est un fichier PNG
        if os.path.isfile(source_file) and filename.lower().endswith(f'.{ext}'):
            # Copie le fichier vers le dossier de destination
            destination_file = os.path.join(destination_dir, filename)
            shutil.copy(source_file, destination_file)

This code is used to create the modifed images with the purple and the yellow squares used to help model 2 to make the difference between commecial and residential buildings. In the "directories" folder you have the images labelled for model2, then this code generate the txt files using model1, those are put in a another folder with the images. Then those images are modfied and put in a folder where the json files (the labels for model2) can be found. All actions could be done in the same folder but to avoid any data loss I believe it's better to create separate folders.

In [26]:
model1_suburbs = YOLO('./runs/detect/train18/weights/best.pt')
model1_countryside = YOLO('./rns/detect/train47/weights/best.pt')
model1_city = YOLO('./rns/detect/train21/weights/best.pt')
models = [model1_countryside, model1_suburbs, model1_city]

directories = ['./modele2/images_suburbs_prepro', './modele2/images_city_prepro', './modele2/images_countryside_prepro'] 
results_directories = ['./modele2/images_suburbs_m2', './modele2/images_city_m2', './modele2/images_countryside_m2']
outputs_directories = ['./modele2/data_suburbs_m2', './modele2/data_city_m2', './modele2/data_countryside_m2']

### TESTS ###
"""
directories = ['./modele2/img_to_process']
results_directories = ['./modele2/data_for_processing']
outputs_directories = ['./modele2/img_processed']
"""
decoding_of_predictions ={0: 'residential', 1: 'commercial'}

for k, direct in enumerate(directories) : 
    generate_txt(direct, results_directories[k], 'png', models, decoding_of_predictions)
    copy_images(direct, results_directories[k], 'png')
    area_identification_val(results_directories[k], outputs_directories[k], 'png')



## Model 2

Now we have images with modifed hue (purple on commercial buldings and yellow on residential buldings). Those images have been labelled beforehand and are now in a folder with their json file. 

### Suburbs

Once again we have to sort the datas in a way yolo will understand. 

In [ ]:
!labelme2yolo --json_dir ./modele2/data_suburbs_m2

# Chemins des répertoires et fichiers
source = "./modele2/data_suburbs_m2/YOLODataset/"
target_dir = "./modele2/modele2_datasets_suburbs"

labelme_to_yolo(source, target_dir)


In [ ]:

model2_suburbs = YOLO('runs/detect/train34/weights/best.pt')
model2_suburbs.train(data='dataset_model2.yaml', epochs= 200, imgsz=512,mixup=0.7, fliplr=0.5, flipud=0.5, copy_paste=0.5, auto_augment='autoaugment', patience=0)

### City

In [ ]:
!labelme2yolo --json_dir ./modele2/data_city_m2

# Chemins des répertoires et fichiers

source = "./modele2/data_city_m2/YOLODataset/"
target_dir = "./modele2/modele2_datasets_city"

labelme_to_yolo(source, target_dir)

In [ ]:
model2_city = YOLO('runs/detect/train37/weights/best.pt')
model2_city.train(data='dataset_model2.yaml', epochs= 200, imgsz=512,mixup=0.7, fliplr=0.5, flipud=0.5, copy_paste=0.5, auto_augment='autoaugment', patience=0)

### Countryside

In [ ]:
!labelme2yolo --json_dir ./modele2/data_countryside_m2

# Chemins des répertoires et fichiers

source = "./modele2/data_countryside_m2/YOLODataset/"
target_dir = "./modele2/modele2_datasets_countryside"

labelme_to_yolo(source, target_dir)

In [ ]:

model2_countryside = YOLO('runs/detect/train27/weights/best.pt')
model2_countryside.train(data='dataset_model2.yaml', epochs= 200, imgsz=512,mixup=0.7, fliplr=0.5, flipud=0.5, copy_paste=0.5, auto_augment='autoaugment', patience=0)

## Porcédure pour la validation sur les images test

In [ ]:
def validation(models1, models2) :
    ################ Modèle 1: 


    directory1 = './VAL/0-IMG_FOR_VAL'
    results_directory1 = './VAL/1-RES_M1'
    decoding_of_predictions ={0: 'residential', 1: 'commercial'}

    generate_txt(directory1, results_directory1, 'png', models, decoding_of_predictions)

    ################## Création du dossier à partir duquel on forme les images pour le modèle 2
    copy_images('./VAL/0-IMG_FOR_VAL','./VAL/1-RES_M1', 'png')

    ################## A partir de ce dossier, fabrication des images à donner au modèle 2
    output_dir = './VAL/2-ENTRY_M2'

    if os.path.exists(output_dir) : shutil.rmtree(output_dir)
    os.mkdir(output_dir)

    area_identification_val('./VAL/1-RES_M1', output_dir, 'png')
    
    ###################### Modèle 2


    ###################### Prédiction 

    # Decoding according to the .yaml file class names order
    decoding_of_predictions ={0: 'damagedcommercialbuilding', 1: 'undamagedcommercialbuilding', 2: 'undamagedresidentialbuilding', 3: 'damagedresidentialbuilding'}
    directory2 = './VAL/2-ENTRY_M2'
    # Directory to store outputs
    results_directory2 = './VAL/3-FINAL_RES'
    generate_txt(directory2, results_directory2, models2, decoding_of_predictions)
    
    ####################### Mise en ZIP 

    # Define your source directory and the destination where the zip file will be created
    source_dir = './VAL/3-FINAL_RES'
    destination_zip = './VAL/submission'

    # Create a zip file from the directory
    shutil.make_archive(destination_zip, 'zip', source_dir)

    print(f"Directory {source_dir} has been successfully zipped into {destination_zip}.")
    


In [ ]:
model1_suburbs = YOLO('./runs/detect/train18/weights/best.pt')
model1_countryside = YOLO('./rns/detect/train47/weights/best.pt')
model1_city = YOLO('./rns/detect/train21/weights/best.pt')
models1 = [model1_countryside, model1_suburbs, model1_city]

model2_suburbs = YOLO('runs/detect/train34/weights/best.pt')
model2_city = YOLO('runs/detect/train37/weights/best.pt')
model2_countryside = YOLO('runs/detect/train27/weights/best.pt')
models2 = [model2_countryside, model2_suburbs, model2_city]
validation(models1, models2)